In [1]:
import numpy as np
import pandas as pd
import gc
#from pandas_profiling import ProfileReport

df_mod = pd.read_csv(r'C:\Users\conno\Documents\School\Brazil\MAI_5003\Projetos\Home_Credit\Python\Feature_Engineering\modelling_table_2.csv')
df_bur =  pd.read_csv(r'C:\Users\conno\Documents\School\Brazil\MAI_5003\Projetos\Home_Credit\Data\bureau.csv')
df_prev = pd.read_csv(r'C:\Users\conno\Documents\School\Brazil\MAI_5003\Projetos\Home_Credit\Data\previous_application.csv')

In [2]:
# Drop the index column
df_mod = df_mod.drop(df_mod.columns[0], axis=1)

In [3]:
# Create copies of the original datasets
df_bur_add = df_bur
df_mod_add = df_mod
df_prev_add = df_prev

In [4]:
# Add a column for the number of previous loans in the last month, 3 months, 6 months, 1 year
### ~~~ LAST MONTH ~~~ ### 
df_bur_l1m = df_bur_add[df_bur_add.DAYS_CREDIT > -31].groupby('SK_ID_CURR',as_index=False)['SK_ID_BUREAU'].count().rename(columns = {'SK_ID_BUREAU': 'num_loans_l1m'})
# Join to our original dataset
df_mod_add = df_mod_add.merge(df_bur_l1m, on = 'SK_ID_CURR', how = 'left')
# Replace the false conditions with 0
df_mod_add['num_loans_l1m'] = df_mod_add['num_loans_l1m'].fillna(0)

### ~~~ LAST 3 MONTHS ~~~ ### 
df_bur_l3m = df_bur_add[df_bur_add.DAYS_CREDIT > -91].groupby('SK_ID_CURR',as_index=False)['SK_ID_BUREAU'].count().rename(columns = {'SK_ID_BUREAU': 'num_loans_l3m'})
# Join to our original dataset
df_mod_add = df_mod_add.merge(df_bur_l3m, on = 'SK_ID_CURR', how = 'left')
# Replace the false conditions with 0
df_mod_add['num_loans_l3m'] = df_mod_add['num_loans_l3m'].fillna(0)

### ~~~ LAST 6 MONTHS ~~~ ### 
df_bur_l6m = df_bur_add[df_bur_add.DAYS_CREDIT > -181].groupby('SK_ID_CURR',as_index=False)['SK_ID_BUREAU'].count().rename(columns = {'SK_ID_BUREAU': 'num_loans_l6m'})
# Join to our original dataset
df_mod_add = df_mod_add.merge(df_bur_l6m, on = 'SK_ID_CURR', how = 'left')
# Replace the false conditions with 0
df_mod_add['num_loans_l6m'] = df_mod_add['num_loans_l6m'].fillna(0)

### ~~~ LAST 12 MONTHS ~~~ ### 
df_bur_l12m = df_bur_add[df_bur_add.DAYS_CREDIT > -366].groupby('SK_ID_CURR',as_index=False)['SK_ID_BUREAU'].count().rename(columns = {'SK_ID_BUREAU': 'num_loans_l12m'})
# Join to our original dataset
df_mod_add = df_mod_add.merge(df_bur_l12m, on = 'SK_ID_CURR', how = 'left')
# Replace the false conditions with 0
df_mod_add['num_loans_l12m'] = df_mod_add['num_loans_l12m'].fillna(0)

# Check correlation to make sure the code was correct (should not = 1)

#df_mod_add['num_loans_l3m'].corr(df_mod_add['num_loans_l1m'])
#df_mod_add['num_loans_l6m'].corr(df_mod_add['num_loans_l3m'])
#df_mod_add['num_loans_l12m'].corr(df_mod_add['num_loans_l6m'])

In [21]:
# Calculate the current length left on the loan by dividing the current amount of credit by the loan annuity
### Credit / Annuity ### 
df_mod_add['remaining_pmts'] = df_mod_add['AMT_CREDIT'] / df_mod_add['AMT_ANNUITY']

In [6]:
# Loan to Income Ratio
df_mod_add['credit_income_ratio'] = df_mod_add['AMT_CREDIT'] / df_mod_add['AMT_INCOME_TOTAL']

In [7]:
#df_mod_add.drop(columns=['sum_tot_credit_debt'])
# Total Credit Debt
df_bur_tot_debt = df_bur_add.groupby('SK_ID_CURR',as_index=False)['AMT_CREDIT_SUM_DEBT'].sum().rename(columns = {'AMT_CREDIT_SUM_DEBT': 'sum_tot_credit_debt'})
# Join to our original dataset
df_mod_add = df_mod_add.merge(df_bur_tot_debt, on = 'SK_ID_CURR', how = 'left')
# Replace the missings with 0
df_mod_add['sum_tot_credit_debt'] = df_mod_add['sum_tot_credit_debt'].fillna(0)

In [8]:
# Total Credit Limit
df_bur_tot_lim = df_bur_add.groupby('SK_ID_CURR',as_index=False)['AMT_CREDIT_SUM'].sum().rename(columns = {'AMT_CREDIT_SUM': 'sum_tot_credit_lim'})
# Join to our original dataset
df_mod_add = df_mod_add.merge(df_bur_tot_lim, on = 'SK_ID_CURR', how = 'left')
# Replace the missings with 0
df_mod_add['sum_tot_credit_lim'] = df_mod_add['sum_tot_credit_lim'].fillna(0)

In [9]:
# Total Credit Debt / Limit Ratio
df_mod_add['tot_debt_lim_ratio'] = df_mod_add['sum_tot_credit_debt'] / df_mod_add['sum_tot_credit_lim']
# Fill NaNs (no limit, no debt, with 0s)
df_mod_add['tot_debt_lim_ratio'] = df_mod_add['tot_debt_lim_ratio'].fillna(0)

# Fill infinites (debt & no limit, with 1s)
df_mod_add['tot_debt_lim_ratio'].replace(np.inf, 1,inplace=True)

In [10]:
# HomeCredit Loan / Limit Ratio
df_mod_add['loan_tot_lim_ratio'] = df_mod_add['AMT_CREDIT'] / df_mod_add['sum_tot_credit_lim']
# Fill infinites (No previous limit, with 1s)
df_mod_add['loan_tot_lim_ratio'].replace(np.inf, 1,inplace=True)

In [11]:
# Boolean value to represent a previous customer with Home Credit who has paid off all of their loans on time

# Boolean for whether the customer is a previously approved customer
df_mod_add['prev_ap_cust'] = np.minimum(1,df_mod_add['cnt_approved_contract_previous'])

# Boolean for whether the customer paid off their previous Home Credit loans appropriately
# These three attributes measure whether the customer was past due:
# For Credit Cards: max_credit_card_sk_dpd_def_previous - Maximum days past due with tolerance on credit card
# For Installments: max_installments_gap_days_payment_previous - Maximum installments gap (when whas paid versus when was supposed to be payd) in days
# Give 15 grace days here:
df_mod_add['max_installments_gap_days_payment_previous_grace'] = df_mod_add['max_installments_gap_days_payment_previous']
df_mod_add.loc[df_mod_add['max_installments_gap_days_payment_previous'] < 46, 'max_installments_gap_days_payment_previous_grace'] = 0

# For POS Cash: max_pos_cash_sk_dpd_previous - Maximum days past due on pos cash

# If all of these are 0, then the customer did not have any payment problems
df_mod_add['no_prev_hc_bad_pmt'] = 1 - np.minimum(df_mod_add[['max_credit_card_sk_dpd_def_previous','max_installments_gap_days_payment_previous_grace','max_pos_cash_sk_dpd_previous']].max(axis=1),1)
#np.maximum(df_mod_add['max_credit_card_sk_dpd_def_previous'],df_mod_add['max_installments_gap_days_payment_previous'],df_mod_add['max_pos_cash_sk_dpd_previous'])

df_mod_add['prev_hc_good_pmt'] = np.minimum(df_mod_add['prev_ap_cust'],df_mod_add['no_prev_hc_bad_pmt'])
df_mod_add = df_mod_add.drop(columns=['no_prev_hc_bad_pmt','max_installments_gap_days_payment_previous_grace'])

In [12]:
# Amount already paid on the loan
df_mod_add['amt_paid_down'] = df_mod_add['AMT_GOODS_PRICE'] - df_mod_add['AMT_CREDIT']

In [13]:
def iv_woe(data, target, bins=10, show_woe=False):
    
    #Empty Dataframe
    newDF,woeDF = pd.DataFrame(), pd.DataFrame()
    
    #Extract Column Names
    cols = data.columns
    
    #Run WOE and IV on all the independent variables
    for ivars in cols[~cols.isin([target])]:
        if (data[ivars].dtype.kind in 'bifc') and (len(np.unique(data[ivars]))>10):
            binned_x = pd.qcut(data[ivars], bins,  duplicates='drop')
            d0 = pd.DataFrame({'x': binned_x, 'y': data[target]})
        else:
            d0 = pd.DataFrame({'x': data[ivars], 'y': data[target]})
        d = d0.groupby("x", as_index=False).agg({"y": ["count", "sum"]})
        d.columns = ['Cutoff', 'N', 'Events']
        d['% of Events'] = np.maximum(d['Events'], 0.5) / d['Events'].sum()
        d['Non-Events'] = d['N'] - d['Events']
        d['% of Non-Events'] = np.maximum(d['Non-Events'], 0.5) / d['Non-Events'].sum()
        d['WoE'] = np.log(d['% of Events']/d['% of Non-Events'])
        d['IV'] = d['WoE'] * (d['% of Events'] - d['% of Non-Events'])
        d.insert(loc=0, column='Variable', value=ivars)
        #print("Information value of " + ivars + " is " + str(round(d['IV'].sum(),6)))
        temp =pd.DataFrame({"Variable" : [ivars], "IV" : [d['IV'].sum()]}, columns = ["Variable", "IV"])
        newDF=pd.concat([newDF,temp], axis=0)
        woeDF=pd.concat([woeDF,d], axis=0)

        #Show WOE Table
        if show_woe == True:
            print(d)
    return newDF, woeDF

In [14]:
#from sklearn.cluster import KMeans

# Calculate the mean Target of the 500 nearest neighbors using only the highest IV attributes
# Attributes to be used in KNN
x_df_mod_knn = df_mod_add[['EXT_SOURCE_3','EXT_SOURCE_1','EXT_SOURCE_2','remaining_pmts']]

from sklearn.neighbors import NearestNeighbors
neigh = NearestNeighbors(n_neighbors=500)
neigh.fit(x_df_mod_knn)

NearestNeighbors(n_neighbors=500)

In [15]:
# Loop through each value in the dataframe and find the mean target for each value
df_mod_add['mean_knn_target'] = -9

In [16]:
# Progress bar
import time, sys
from IPython.display import clear_output

def update_progress(progress):
    bar_length = 20
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1

    block = int(round(bar_length * progress))

    clear_output(wait = True)
    text = "Progress: [{0}] {1:.1f}%".format( "#" * block + "-" * (bar_length - block), progress * 100)
    print(text)

In [17]:
# Calculate the mean Target of the 500 nearest neighbors using only the highest IV attributes
# Attributes to be used in KNN
x_df_mod_knn = df_mod_add[['EXT_SOURCE_3','EXT_SOURCE_1','EXT_SOURCE_2','remaining_pmts']]

for i,r in x_df_mod_knn.iterrows():
    nn = neigh.kneighbors([r],return_distance=False)
    nn_df = df_mod_add.iloc[list(nn[0,:])]
    df_mod_add.loc[i, 'mean_knn_target'] = nn_df['TARGET'].mean()
    
    update_progress(i / x_df_mod_knn.shape[0])

update_progress(1)

Progress: [####################] 100.0%


In [20]:
#df_mod_add['mean_knn_target'].value_counts()
#df_mod_add['mean_knn_target'].describe()

count    307511.000000
mean          0.077868
std           0.066178
min           0.000000
25%           0.030000
50%           0.058000
75%           0.104000
max           0.446000
Name: mean_knn_target, dtype: float64

In [23]:
iv, woe = iv_woe(data=df_mod_add,target='TARGET',bins=10,show_woe=False)

iv = iv.sort_values(by='IV',ascending=False)
iv.to_csv('Information_Value_NewTable2.csv')

In [24]:
df_mod_add.to_csv('modeling_table_3.csv')